In [11]:
import pandas as pd
from tqdm import tqdm
import os
import numpy as np
import matplotlib.pyplot as plt

import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import yfinance as yf
import requests

In [2]:
path = '/Users/matthesfogtmann/Downloads/SEC data/'

# importing and restructuring all the finance data

In [221]:
def getDir(path):
    lst = []
    for i in os.listdir(path):
        if len(i)==6 and "q" in i:
            lst.append(i)
    return lst

def getFinacialData(filename=path+'/2022q4'):
    data = {i:pd.read_csv(f"{filename}{i}.txt",sep="\t") for i in ["tag","num","pre","sub"]}
    
    #drop things
    data["tag"].drop(columns=["version","custom","abstract","datatype","iord","crdr"],inplace=True)
    data["num"].drop(columns=["footnote"],inplace=True)
    data["pre"].drop(columns=["version","negating"],inplace=True)
    data["sub"].drop(columns=['stprba', 'zipba',
       'bas1', 'bas2', 'baph', 'countryma', 'stprma', 'cityma', 'zipma',
       'mas1', 'mas2', 'ein'],inplace=True)
    
    #make mapper for company names
    
    def mapper(colname):
        return {i:j for i,j in zip(data["sub"]["adsh"],data["sub"][colname])}
    
    def inverseMapper(colname):
        return {j:i for i,j in zip(data["sub"]["adsh"],data["sub"][colname])}
    
    data["num"]["name"] = data["num"]["adsh"].map(mapper("name"))
    data["pre"]["name"] = data["pre"]["adsh"].map(mapper("name"))
    
    data["num"].drop(columns=["version","coreg","adsh"],inplace=True)
    data["pre"].drop(columns=["adsh","plabel","inpth","rfile"],inplace=True)
    
    
    #get some sampels of company
    names = sorted(list(data["num"]["name"].unique()))[:3]
    stmts = data["pre"]["stmt"].unique()
    
    print(names)
    def f(data, names):
        dic = dict()
        for name in tqdm(names):
            #get dfs where name is equal to name
            df = data["num"][data["num"]["name"]==name].copy()
            df_stmt = data["pre"][data["pre"]["name"]==name]
            
            #creat of company dic to load finacial data onto
            dic[name] = dict()
            
            # creat stmts for each company
            for stmt in stmts:
                dic[name][stmt] = dict()
                
            # fill more data
            
            for tag in set(df["tag"]):
                df_stmt_temp = df_stmt[df_stmt["tag"]==tag]
                for stmt in set(df_stmt_temp["stmt"]):
                    df_temp = df[df["tag"]==tag]
                    df_temp = df_temp[df_temp["ddate"]==max(df_temp["ddate"])]
                    dic[name][stmt][list(df_temp["tag"])[0]]=df_temp["value"].values[0]
            
               
        return dic
    return f(data, names)


def combineReports(reports):
    reformed_dic = {}

    for report, level1 in reports.items():
        for company,level2 in level1.items():
            for stmt, level3 in level2.items():
                for tag, value in level3.items():
                    reformed_dic[(company,stmt,tag)] = {report:value}
                    
    return pd.DataFrame(reformed_dic)


def getTicker(company_name):
    
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']
    return company_code

@np.vectorize
def timeString2float(x="2020q2"):
    lst = x.split("q")
    return int(lst[0])+(int(lst[1])-1)*0.25

In [180]:
reports = {i : getFinacialData(path+i+"/") for i in getDir(path)}

  0%|          | 0/3 [00:00<?, ?it/s]

['1 800 FLOWERS COM INC', '10X CAPITAL VENTURE ACQUISITION CORP. II', '10X CAPITAL VENTURE ACQUISITION CORP. III']


  0%|          | 0/3 [00:00<?, ?it/s]

['1 800 FLOWERS COM INC', '10X CAPITAL VENTURE ACQUISITION CORP. II', '10X CAPITAL VENTURE ACQUISITION CORP. III']


  0%|          | 0/3 [00:00<?, ?it/s]

['1 800 FLOWERS COM INC', '10X CAPITAL VENTURE ACQUISITION CORP. II', '10X CAPITAL VENTURE ACQUISITION CORP. III']


100%|██████████| 3/3 [00:00<00:00,  4.54it/s]
<ipython-input-174-154d8f624564>:9: DtypeWarning:

Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.

  0%|          | 0/3 [00:00<?, ?it/s]

['1 800 FLOWERS COM INC', '10X CAPITAL VENTURE ACQUISITION CORP. II', '10X GENOMICS, INC.']


100%|██████████| 3/3 [00:00<00:00,  3.93it/s]


In [222]:
combineReports(reports)

1 800 FLOWERS COM INC                             \
                            BS                              
       AdditionalPaidInCapital OtherLiabilitiesNoncurrent   
2022q1             377234000.0                 22112000.0   
2022q3                     NaN                        NaN   
2022q2                     NaN                        NaN   

                                                                    \
                                                                     
       OperatingLeaseLiabilityNoncurrent AccruedLiabilitiesCurrent   
2022q1                       128620000.0               279345000.0   
2022q3                               NaN                       NaN   
2022q2                               NaN                       NaN   

                                                                  \
                                                                   
       PreferredStockSharesIssued PreferredStockSharesAuthorized   
2022q1                        0.0                     10000000.0   
2022q3                        NaN                            NaN   
2022q2                        NaN                            NaN   

                                                            \
                                                             
       DeferredIncomeTaxLiabilitiesNet LongTermDebtCurrent   
2022q1                      32856000.0          20000000.0   
2022q3                             NaN                 NaN   
2022q2                             NaN                 NaN   

                                               \
                                                
       PreferredStockParOrStatedValuePerShare   
2022q1                                   0.01   
2022q3                                    NaN   
2022q2                                    NaN   

                                              ...  \
                                              ...   
       CashAndCashEquivalentsAtCarryingValue  ...   
2022q1                           271068000.0  ...   
2022q3                                   NaN  ...   
2022q2                                   NaN  ...   

                 10X GENOMICS, INC.               \
                                 BS                
       PropertyPlantAndEquipmentNet InventoryNet   
2022q1                  169492000.0   59966000.0   
2022q3                          NaN          NaN   
2022q2                          NaN          NaN   

                                                              \
                                                               
       RetainedEarningsAccumulatedDeficit LiabilitiesCurrent   
2022q1                       -863321000.0        110357000.0   
2022q3                                NaN                NaN   
2022q2                                NaN                NaN   

                                                                               \
                                                                                
       AdditionalPaidInCapitalCommonStock PrepaidExpenseAndOtherAssetsCurrent   
2022q1                       1.680865e+09                          13896000.0   
2022q3                                NaN                                 NaN   
2022q2                                NaN                                 NaN   

                                                              \
                                                               
       EmployeeRelatedLiabilitiesCurrent PreferredStockValue   
2022q1                        31626000.0                 0.0   
2022q3                               NaN                 NaN   
2022q2                               NaN                 NaN   

                                                       
                                                   CP  
       LicenseFeesAccruedNoncurrent EntityPublicFloat  
2022q1                    5814000.0      1.870000e+10  
2022q3                         

# company data

In [231]:
filename=path+'2022q4/'

data = {i:pd.read_csv(f"{filename}{i}.txt",sep="\t") for i in ["sub"]}

#drop things
data["sub"].drop(columns=['stprba', 'zipba', 'bas2', 'baph', 'countryma', 'stprma', 'cityma', 'zipma',
                          'mas1', 'mas2', 'ein'],inplace=True)





In [228]:
data = {i:pd.read_csv(f"{filename}{i}.txt",sep="\t") for i in ["tag","num","pre","sub"]}

In [233]:
data["sub"]

,adsh,cik,name,sic,countryba,cityba,bas1,countryinc,stprinc,former,...,period,fy,fp,filed,accepted,prevrpt,detail,instance,nciks,aciks
0,0000014707-22-000072,14707,CALERES INC,3140.0,US,ST LOUIS,8300 MARYLAND AVE,US,NY,BROWN SHOE CO INC,...,20221031.0,2022.0,Q3,20221206,2022-12-06 14:50:00.0,0,1,cal-20221029x10q_htm.xml,1,NaN
1,0000014846-22-000059,14846,BRT APARTMENTS CORP.,6798.0,US,GREAT NECK,60 CUTTER MILL RD,US,NY,BRT REALTY TRUST,...,20221130.0,NaN,NaN,20221206,2022-12-06 16:16:00.0,0,0,brt-20221206_htm.xml,1,NaN
2,0000018349-22-000191,18349,SYNOVUS FINANCIAL CORP,6021.0,US,COLUMBUS,1111 BAY AVENUE,US,GA,CB&T BANCSHARES INC,...,20221130.0,NaN,NaN,20221206,2022-12-06 16:16:00.0,0,0,syn-20221206_htm.xml,1,NaN
3,0000035527-22-000248,35527,FIFTH THIRD BANCORP,6022.0,US,CINCINNATI,38 FOUNTAIN SQ PLZ,US,OH,NaN,...,20221130.0,NaN,NaN,20221206,2022-12-06 17:01:00.0,0,0,fitb-20221206_htm.xml,1,NaN
4,0000041719-22-000063,41719,GLATFELTER CORP,2621.0,US,CHARLOTTE,4350 CONGRESS STREET,US,PA,GLATFELTER P H CO,...,20221130.0,NaN,NaN,20221206,2022-12-06 16:47:00.0,0,0,glt-20221202_htm.xml,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23938,0001851734-22-000581,1386570,CHROMADEX CORP.,2833.0,US,LOS ANGELES,10900 WILSHIRE BLVD,US,DE,"CODY RESOURCES, INC.",...,20220930.0,NaN,NaN,20221003,2022-10-03 06:43:00.0,0,0,cdxc20220930_8k_htm.xml,1,NaN
23939,0001852117-22-000371,1852117,ROCKLEY PHOTONICS HOLDINGS LTD,3674.0,KY,GEORGE TOWN,"HARBOUR PLACE, 2ND FLOOR",KY,NaN,NaN,...,20220930.0,NaN,NaN,20221003,2022-10-03 16:58:00.0,0,0,rkly-20220930_htm.xml,1,NaN
23940,0001854270-22-000033,1854270,"SENTI BIOSCIENCES, INC.",2836.0,US,SOUTH SAN FRANCISCO,"2 CORPORATE DRIVE, FIRST FLOOR",US,DE,"SENTI BIOSCIENCES, INC.",...,20220930.0,NaN,NaN,20221003,2022-09-30 21:59:00.0,0,0,snti-20220930_htm.xml,1,NaN
23941,0001867096-22-000111,1867096,"XERIS BIOPHARMA HOLDINGS, INC.",2834.0,US,CHICAGO,"180 N. LASALLE STREET, SUITE 1600",US,DE,NaN,...,20220930.0,NaN,NaN,20221003,2022-10-03 16:03:00.0,0,0,xers-20220929_htm.xml,1,NaN


In [60]:
df = pd.read_csv("/Users/matthesfogtmann/Downloads/SEC data/2022q2/SUB.txt",sep="\t")

In [63]:
df["bas1"].iloc[0]

'5260 CALIFORNIA AVENUE'

In [64]:
import geopandas

In [75]:
df = geopandas.tools.geocode('5260 CALIFORNIA AVENUE')

In [105]:
df["long"] = df["geometry"].apply(lambda a: a.centroid.x)
df["lat"] = df["geometry"][0].centroid.y

In [102]:
df

,geometry,address,lat,long
0,POINT (-117.04063 32.77397),"5260, Guessman Avenue, 91942, Guessman Avenue,...",-117.040634,32.773968


In [ ]:
locator = Nominatim(user_agent=”myGeocoder”)
location = locator.geocode(“Champ de Mars, Paris, France”)

In [124]:
import plotly.express as px

fig = px.scatter_geo(df, lat="lat",lon="long",size=[10])
#fig.show()

In [93]:
df["geometry"][0].centroid.x

-117.040634

In [34]:
getTicker("10X GENOMICS, INC.")

'TXG'

In [41]:
report

,BS,CF,IS,EQ,CP,CI,UN
1 800 FLOWERS COM INC,"{'AdditionalPaidInCapital': 378741000.0, 'Othe...","{'IncreaseDecreaseInInventories': 51928000.0, ...",{'WeightedAverageNumberOfSharesOutstandingBasi...,{'StockIssuedDuringPeriodValueShareBasedCompen...,{},{},{}
10X CAPITAL VENTURE ACQUISITION CORP. II,"{'AccountsPayableCurrent': 255700.0, 'Liabilit...",{'CashAndCashEquivalentsPeriodIncreaseDecrease...,{'RelatedPartyTransactionSellingGeneralAndAdmi...,"{'NetIncomeLoss': -1906041.0, 'StockIssuedDuri...",{},{},{}
10X CAPITAL VENTURE ACQUISITION CORP. III,"{'AccountsPayableCurrent': 103609.0, 'Liabilit...",{'CashAndCashEquivalentsPeriodIncreaseDecrease...,{'RelatedPartyTransactionSellingGeneralAndAdmi...,"{'NetIncomeLoss': -347368.0, 'StockIssuedDurin...",{},{},{}


In [116]:
report = getFinacialData(path+"2022q2"+"/")

100%|██████████| 3/3 [00:00<00:00,  4.58it/s]


In [123]:
report

,BS,CF,IS,EQ,CP,CI,UN
1 800 FLOWERS COM INC,"{'AdditionalPaidInCapital': 378741000.0, 'Othe...","{'IncreaseDecreaseInInventories': 51928000.0, ...",{'WeightedAverageNumberOfSharesOutstandingBasi...,{'StockIssuedDuringPeriodValueShareBasedCompen...,{},{},{}
10X CAPITAL VENTURE ACQUISITION CORP. II,"{'AccountsPayableCurrent': 255700.0, 'Liabilit...",{'CashAndCashEquivalentsPeriodIncreaseDecrease...,{'RelatedPartyTransactionSellingGeneralAndAdmi...,"{'NetIncomeLoss': -1906041.0, 'StockIssuedDuri...",{},{},{}
10X CAPITAL VENTURE ACQUISITION CORP. III,"{'AccountsPayableCurrent': 103609.0, 'Liabilit...",{'CashAndCashEquivalentsPeriodIncreaseDecrease...,{'RelatedPartyTransactionSellingGeneralAndAdmi...,"{'NetIncomeLoss': -347368.0, 'StockIssuedDurin...",{},{},{}


In [68]:
newreport = getFinacialData(path+"2022q2"+"/")

100%|██████████| 20/20 [00:04<00:00,  4.06it/s]


In [120]:
combineReports(report).T

1 800 FLOWERS COM INC      \
                                                                      BS  CF   
AdditionalPaidInCapital                                      378741000.0 NaN   
OtherLiabilitiesNoncurrent                                    21802000.0 NaN   
OperatingLeaseLiabilityNoncurrent                            125831000.0 NaN   
AccruedLiabilitiesCurrent                                    176551000.0 NaN   
PreferredStockSharesIssued                                           0.0 NaN   
...                                                                  ...  ..   
PaymentsToAcquireRestrictedInvestments                               NaN NaN   
DeferredUnderwritingCommissionsIncurredButNotYe...                   NaN NaN   
RepaymentsOfRelatedPartyDebt                                         NaN NaN   
TemporaryEquityAccretionToRedemptionValue                            NaN NaN   
AdjustmentsToAdditionalPaidInCapitalStockIssued...                   NaN NaN   

                                                                        \
                                                    IS  EQ  CP  CI  UN   
AdditionalPaidInCapital                            NaN NaN NaN NaN NaN   
OtherLiabilitiesNoncurrent                         NaN NaN NaN NaN NaN   
OperatingLeaseLiabilityNoncurrent                  NaN NaN NaN NaN NaN   
AccruedLiabilitiesCurrent                          NaN NaN NaN NaN NaN   
PreferredStockSharesIssued                         NaN NaN NaN NaN NaN   
...                                                 ..  ..  ..  ..  ..   
PaymentsToAcquireRestrictedInvestments             NaN NaN NaN NaN NaN   
DeferredUnderwritingCommissionsIncurredButNotYe... NaN NaN NaN NaN NaN   
RepaymentsOfRelatedPartyDebt                       NaN NaN NaN NaN NaN   
TemporaryEquityAccretionToRedemptionValue          NaN NaN NaN NaN NaN   
AdjustmentsToAdditionalPaidInCapitalStockIssued... NaN NaN NaN NaN NaN   

                                                   10X CAPITAL VENTURE ACQUISITION CORP. II  \
                                                                                         BS   
AdditionalPaidInCapital                                                                 NaN   
OtherLiabilitiesNoncurrent                                                              NaN   
OperatingLeaseLiabilityNoncurrent                                                       NaN   
AccruedLiabilitiesCurrent                                                         2475592.0   
PreferredStockSharesIssued                                                              0.0   
...                                                                                     ...   
PaymentsToAcquireRestrictedInvestments                                                  NaN   
DeferredUnderwritingCommissionsIncurredButNotYe...                                      NaN   
RepaymentsOfRelatedPartyDebt                                                            NaN   
TemporaryEquityAccretionToRedemptionValue                                               NaN   
AdjustmentsToAdditionalPaidInCapitalStockIssued...                                      NaN   

                                                            ...              \
                                                    CF  IS  ...  CP  CI  UN   
AdditionalPaidInCapital                            NaN NaN  ... NaN NaN NaN   
OtherLiabilitiesNoncurrent                         NaN NaN  ... NaN NaN NaN   
OperatingLeaseLiabilityNoncurrent                  NaN NaN  ... NaN NaN NaN   
AccruedLiabilitiesCurrent                          NaN NaN  ... NaN NaN NaN   
PreferredStockSharesIssued                         NaN NaN  ... NaN NaN NaN   
...                                                 ..  ..  ...  ..  ..  ..   
PaymentsToAcquireRestrictedInvestments             NaN NaN  ... NaN NaN NaN   
DeferredUnderwritingCommissionsIncurredButNotYe... NaN NaN  ... NaN NaN NaN   
RepaymentsOfRelatedPa

In [ ]:
def getTicker(company_name):
    yfinance = "https://query2.finance.yahoo.com/v1/finance/search"
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'
    params = {"q": company_name, "quotes_count": 1, "country": "United States"}

    res = requests.get(url=yfinance, params=params, headers={'User-Agent': user_agent})
    data = res.json()

    company_code = data['quotes'][0]['symbol']
    return company_code

In [49]:
getTicker("2U, INC.")

<Response [200]>


'TWOU'

In [37]:
reports = {i : combineReports(getFinacialData(path+i+"/")) for i in getDir(path)}

100%|██████████| 3/3 [00:00<00:00,  4.79it/s]
<ipython-input-36-060c1b2ee209>:9: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  data = {i:pd.read_csv(f"{filename}{i}.txt",sep="\t") for i in ["tag","num","pre","sub"]}
100%|██████████| 3/3 [00:00<00:00,  3.96it/s]


In [155]:
reports

{'2022q2':                                               AdditionalPaidInCapital  \
 1 800 FLOWERS COM INC                     BS              378741000.0   
                                           CF                      NaN   
                                           IS                      NaN   
                                           EQ                      NaN   
                                           CP                      NaN   
                                           CI                      NaN   
                                           UN                      NaN   
 10X CAPITAL VENTURE ACQUISITION CORP. II  BS                      NaN   
                                           CF                      NaN   
                                           IS                      NaN   
                                           EQ                      NaN   
                                           CP                      NaN   
                            

In [152]:
dataset = combineReports(reports)

2022q2
2022q4
2022q3
2022q1
{'AdditionalPaidInCapital': {'2022q2': 1 800 FLOWERS COM INC                      BS    378741000.0
                                           CF            NaN
                                           IS            NaN
                                           EQ            NaN
                                           CP            NaN
                                           CI            NaN
                                           UN            NaN
10X CAPITAL VENTURE ACQUISITION CORP. II   BS            NaN
                                           CF            NaN
                                           IS            NaN
                                           EQ            NaN
                                           CP            NaN
                                           CI            NaN
                                           UN            NaN
10X CAPITAL VENTURE ACQUISITION CORP. III  BS            0.0
                  

In [153]:
dataset

AdditionalPaidInCapital                              \
         1 800 FLOWERS COM INC                               
                            BS  CF  CI  CP  EQ  IS  SI  UN   
2022q2             378741000.0 NaN NaN NaN NaN NaN NaN NaN   
2022q4             381440000.0 NaN NaN NaN NaN NaN NaN NaN   
2022q3             379885000.0 NaN NaN NaN NaN NaN NaN NaN   
2022q1             377234000.0 NaN NaN NaN NaN NaN NaN NaN   

                                                     ...  \
       10X CAPITAL VENTURE ACQUISITION CORP. II      ...   
                                             BS  CF  ...   
2022q2                                      NaN NaN  ...   
2022q4                                      NaN NaN  ...   
2022q3                                      NaN NaN  ...   
2022q1                                      0.0 NaN  ...   

                   LicenseFeesAccruedNoncurrent                             \
       10X CAPITAL VENTURE ACQUISITION CORP. II         10X GENOMICS, INC.   
                                             UN  BS  CP                 IS   
2022q2                                      NaN NaN NaN                NaN   
2022q4                                      NaN NaN NaN                NaN   
2022q3                                      NaN NaN NaN                NaN   
2022q1                                      NaN NaN NaN                NaN   

                                       
                                       
        EQ  CF  CI  UN         BS  CP  
2022q2 NaN NaN NaN NaN        NaN NaN  
2022q4 NaN NaN NaN NaN        NaN NaN  
2022q3 NaN NaN NaN NaN        NaN NaN  
2022q1 NaN NaN NaN NaN  5814000.0 NaN  

[4 rows x 5033 columns]

In [48]:
dataset.to_csv("SECNewDAta.csv")

In [53]:
sec = pd.read_csv("SECNewDAta.csv",index_col=[0,1],header=[0,1])

In [111]:
sec["1 800 FLOWERS COM INC"]

BS  CF  CI  CP  EQ  IS  \
AdditionalPaidInCapital              2022q2  378741000.0 NaN NaN NaN NaN NaN   
                                     2022q4  381440000.0 NaN NaN NaN NaN NaN   
                                     2022q3  379885000.0 NaN NaN NaN NaN NaN   
                                     2022q1  377234000.0 NaN NaN NaN NaN NaN   
OtherLiabilitiesNoncurrent           2022q2   21802000.0 NaN NaN NaN NaN NaN   
...                                                  ...  ..  ..  ..  ..  ..   
CommonStockSharesOutstanding         2022q1          NaN NaN NaN NaN NaN NaN   
ContractWithCustomerLiabilityCurrent 2022q1          NaN NaN NaN NaN NaN NaN   
AdditionalPaidInCapitalCommonStock   2022q1          NaN NaN NaN NaN NaN NaN   
EmployeeRelatedLiabilitiesCurrent    2022q1          NaN NaN NaN NaN NaN NaN   
LicenseFeesAccruedNoncurrent         2022q1          NaN NaN NaN NaN NaN NaN   

                                             SI  UN  
AdditionalPaidInCapital              2022q2 NaN NaN  
                                     2022q4 NaN NaN  
                                     2022q3 NaN NaN  
                                     2022q1 NaN NaN  
OtherLiabilitiesNoncurrent           2022q2 NaN NaN  
...                                          ..  ..  
CommonStockSharesOutstanding         2022q1 NaN NaN  
ContractWithCustomerLiabilityCurrent 2022q1 NaN NaN  
AdditionalPaidInCapitalCommonStock   2022q1 NaN NaN  
EmployeeRelatedLiabilitiesCurrent    2022q1 NaN NaN  
LicenseFeesAccruedNoncurrent         2022q1 NaN NaN  

[519 rows x 8 columns]

In [30]:
df = pd.DataFrame(dataset["1 800 FLOWERS COM INC"]["BS"])

In [33]:
df.reset_index(level=1)

,level_1,BS
IntangibleAssetsNetExcludingGoodwill,2022q2,146641000.0
IntangibleAssetsNetExcludingGoodwill,2022q4,144508000.0
IntangibleAssetsNetExcludingGoodwill,2022q3,145568000.0
IntangibleAssetsNetExcludingGoodwill,2022q1,147178000.0
AssetsCurrent,2022q2,381856000.0
...,...,...
RestrictedCashNoncurrent,2022q1,NaN
CommonStockSharesOutstanding,2022q1,NaN
AdditionalPaidInCapitalCommonStock,2022q1,NaN
AccruedExpensesandOtherLiabilitiesCurrent,2022q1,NaN


In [18]:
reports["2022q2"]["1 800 FLOWERS COM INC"]

KeyError: '1 800 FLOWERS COM INC'

In [108]:
reports["2022q2"]

AccumulatedOtherComprehensiveIncomeLossNetOfTax  \
1 800 FLOWERS COM INC BS                                        -318000.0   
                      CF                                              NaN   
                      IS                                              NaN   
                      EQ                                              NaN   
                      CP                                              NaN   
...                                                                   ...   
2U, INC.              IS                                              NaN   
                      EQ                                              NaN   
                      CP                                              NaN   
                      CI                                              NaN   
                      UN                                              NaN   

                          PrepaidExpenseAndOtherAssetsCurrent  \
1 800 FLOWERS COM INC BS                           33506000.0   
                      CF                                  NaN   
                      IS                                  NaN   
                      EQ                                  NaN   
                      CP                                  NaN   
...                                                       ...   
2U, INC.              IS                                  NaN   
                      EQ                                  NaN   
                      CP                                  NaN   
                      CI                                  NaN   
                      UN                                  NaN   

                          IntangibleAssetsNetExcludingGoodwill  \
1 800 FLOWERS COM INC BS                           146641000.0   
                      CF                                   NaN   
                      IS                                   NaN   
                      EQ                                   NaN   
                      CP                                   NaN   
...                                                        ...   
2U, INC.              IS                                   NaN   
                      EQ                                   NaN   
                      CP                                   NaN   
                      CI                                   NaN   
                      UN                                   NaN   

                          OperatingLeaseLiabilityCurrent  \
1 800 FLOWERS COM INC BS                      12518000.0   
                      CF                             NaN   
                      IS                             NaN   
                      EQ                             NaN   
                      CP                             NaN   
...                                                  ...   
2U, INC.              IS                             NaN   
                      EQ                             NaN   
                      CP                             NaN   
                      CI                             NaN   
                      UN                             NaN   

                          AccruedLiabilitiesCurrent     Goodwill  \
1 800 FLOWERS COM INC BS                176551000.0  213905000.0   
                      CF                        NaN          NaN   
                      IS                        NaN          NaN   
                      EQ                        NaN          NaN   
                      CP                        NaN          NaN   
...                                             ...          ...   
2U, INC.              IS                        NaN          NaN   
                      EQ                        NaN          NaN   
                      CP                        NaN          NaN   
                      CI                        NaN          NaN   
                      UN                        NaN       

In [8]:
reports["2022q2"]

AccumulatedOtherComprehensiveIncomeLossNetOfTax  \
BS 1 800 FLOWERS COM INC                                                            -318000.0   
   10X CAPITAL VENTURE ACQUISITION CORP. II                                               NaN   
   10X CAPITAL VENTURE ACQUISITION CORP. III                                              NaN   
   10X GENOMICS, INC.                                                              -2443000.0   
   111, INC.                                                                       59371000.0   
...                                                                                       ...   
UN 22ND CENTURY GROUP, INC.                                                               NaN   
   23ANDME HOLDING CO.                                                                    NaN   
   26 CAPITAL ACQUISITION CORP.                                                           NaN   
   2SEVENTY BIO, INC.                                                                     NaN   
   2U, INC.                                                                               NaN   

                                              PrepaidExpenseAndOtherAssetsCurrent  \
BS 1 800 FLOWERS COM INC                                               33506000.0   
   10X CAPITAL VENTURE ACQUISITION CORP. II                                   NaN   
   10X CAPITAL VENTURE ACQUISITION CORP. III                                  NaN   
   10X GENOMICS, INC.                                                  17762000.0   
   111, INC.                                                          242199000.0   
...                                                                           ...   
UN 22ND CENTURY GROUP, INC.                                                   NaN   
   23ANDME HOLDING CO.                                                        NaN   
   26 CAPITAL ACQUISITION CORP.                                               NaN   
   2SEVENTY BIO, INC.                                                         NaN   
   2U, INC.                                                                   NaN   

                                              IntangibleAssetsNetExcludingGoodwill  \
BS 1 800 FLOWERS COM INC                                               146641000.0   
   10X CAPITAL VENTURE ACQUISITION CORP. II                                    NaN   
   10X CAPITAL VENTURE ACQUISITION CORP. III                                   NaN   
   10X GENOMICS, INC.                                                   24764000.0   
   111, INC.                                                             4909000.0   
...                                                                            ...   
UN 22ND CENTURY GROUP, INC.                                                    NaN   
   23ANDME HOLDING CO.                                                         NaN   
   26 CAPITAL ACQUISITION CORP.                                                NaN   
   2SEVENTY BIO, INC.                                                          NaN   
   2U, INC.                                                                    NaN   

                                              OperatingLeaseLiabilityCurrent  \
BS 1 800 FLOWERS COM INC                                          12518000.0   
   10X CAPITAL VENTURE ACQUISITION CORP. II                              NaN   
   10X CAPITAL VENTURE ACQUISITION CORP. III                             NaN   
   10X GENOMICS, INC.                                              7637000.0   
   111, INC.                                                             NaN   
...                                                                      ...   
UN 22ND CENTURY GROUP, INC.                                              NaN   
   23ANDME HOLDING CO.                                                   NaN   
   26 CAPITAL ACQUISITION CORP.                                          NaN   
   2SEVENTY BIO, INC.      

In [109]:
dataset = combineReports(reports)

In [164]:
dataset["1 800 FLOWERS COM INC"]["BS"]

AccumulatedOtherComprehensiveIncomeLossNetOfTax            2022q2     -318000.0
                                                           2022q4     -211000.0
                                                           2022q3     -211000.0
                                                           2022q1     -318000.0
PrepaidExpenseAndOtherAssetsCurrent                        2022q2    33506000.0
                                                                        ...    
TransferOfEquipmentToInventoryFromPropertyAndEquipmentNet  2022q1           NaN
ContractWithCustomerLiabilityCustomerDepositsCurrent       2022q1           NaN
OtherLongTermDebtNoncurrent                                2022q1           NaN
StockBasedCompensation                                     2022q1           NaN
PaymentsToAcquireTradingSecuritiesHeldforinvestment        2022q1           NaN
Name: BS, Length: 3417, dtype: float64

In [111]:
dataset.to_csv('test.csv')

In [82]:
stmt_dic = {"BS" : "Balance Sheet", "IS" : "Income Statement", "CF" : "Cash Flow", "EQ" : "Equity",
 "CI": "Comprehensive Income", "UN" : "Unclassifiable Statement", "CP" :"Cover Page"}

In [90]:
def plotCompanyTag(companies=["FLOWERS", "force"],tags="Assets"):

    def companyFinder(approxName,df):
        approxName = approxName.lower()
        for i in set(np.array(list(df.index))[:,0]):
            if approxName in i.lower():
                return i
    @np.vectorize
    def timeString2date(x="2020q2"):
        lst = x.split("q")
        num = int(lst[0])+(int(lst[1])-1)*0.25
        labels = f"{x[:5]} {x[:-1]}"
        return num,labels
    
    fig, ax = plt.subplots(1,len(tags))
    
    for i in range(len(tags)):

        all_companies = set(np.array(list(df.index))[:,0])
        for company in companies:
            if company not in all_companies:
                company = companyFinder(company,df)
                y = df.loc[company][tags[i]].values
                #print(df.loc[company][tags[i]].index)
                x,labels = timeString2date(df.loc[company][tags[i]].index)
                #print(x)
                
               
                sort_index = np.argsort(x)

                x = x[sort_index]
                y = y[sort_index]
                print(x)

                ax[i].plot(x,y,label=company)
                ax[i].set_xticklabels(labels)
                ax[i].legend()
                ax[i].set_ylabel(tags[i])

In [113]:
pre = pd.read_csv("/Users/matthesfogtmann/Downloads/SEC data/2022q3/pre.txt",sep="\t")

In [ ]:
num = pd.read_csv("/Users/matthesfogtmann/Downloads/SEC data/2022q3/num.txt",sep="\t")

In [118]:
sub = pd.read_csv("/Users/matthesfogtmann/Downloads/SEC data/2022q3/sub.txt",sep="\t")